In [1]:
using Revise

In [2]:
using Flux

In [3]:
using PaddedViews

In [4]:
using ModelVerification

In [52]:
include("/home/verification/ModelVerification.jl/src/propagate/operators/normalise.jl")

bound_backward (generic function with 1 method)

In [53]:
bn_layer = BatchNorm(2)
last_A = [0.80288410  1.77289701  4.04968071  1.10468459; 
-1.95263970  0.03672504 -0.94798982  2.70319653; 
-1.37702072 -0.42611432  0.75072187 -1.17853343; 
-2.34488750  0.19852549  1.74567282  2.40724754;;; 
0.57746184 -1.12509835 -0.65899199  0.09659922; 
2.02263784 -0.49920616  0.24671881 -1.36043179; 
-0.92502880  0.13907211  0.80686390 -3.31853700; 
0.67951810 -0.25937605 -0.62096345  0.68446565;;;;]
batch_info = nothing
batch_reach, batch_bias, batch_info = bound_backward(bn_layer, last_A, batch_info)
println(batch_reach)
println(batch_bias)

[0.8028800801277876 1.7728881334766937 4.049660434098465 1.1046790590760636; -1.9526299235427618 0.03672485612543106 -0.9479850736138964 2.70318299566221; -1.3770138255462074 -0.4261121865350151 0.7507181112931276 -1.1785275293304183; -2.344875759640336 0.19852449602452993 1.7456640797825003 2.4072354874167017;;; 0.577458948769741 -1.1250927168686509 -0.6589886905653882 0.09659873634797574; 2.022627713076763 -0.49920366057950977 0.24671757473068953 -1.360424978603499; -0.9250241685749053 0.13907141369512796 0.806859860201764 -3.318520384781599; 0.6795146977954626 -0.2593747513585449 -0.6209603409663081 0.684462223024118;;;;]
[0.0]


In [46]:
#= model = Chain(
    Conv((4, 4), 3=>16, stride=2, pad=(1, 1), identity),
    Conv((3, 3), 16=>32, stride=2, pad=(1, 1), identity),
    Conv((5, 5), 32=>64, stride=2, pad=(1, 1), identity),
) =#
bn_layer = BatchNorm(2)
β, γ, μ, σ², ϵ, momentum, affine, track_stats = bn_layer.β, bn_layer.γ, bn_layer.μ, bn_layer.σ², bn_layer.ϵ, bn_layer.momentum, bn_layer.affine, bn_layer.track_stats
channels = 2
β = affine ? β : 1.0
γ = affine ? γ : 0.0
μ = track_stats ? μ : zeros(channels)
σ² = track_stats ? σ² : ones(channels)
last_A = [0.80288410  1.77289701  4.04968071  1.10468459; 
-1.95263970  0.03672504 -0.94798982  2.70319653; 
-1.37702072 -0.42611432  0.75072187 -1.17853343; 
-2.34488750  0.19852549  1.74567282  2.40724754;;; 
0.57746184 -1.12509835 -0.65899199  0.09659922; 
2.02263784 -0.49920616  0.24671881 -1.36043179; 
-0.92502880  0.13907211  0.80686390 -3.31853700; 
0.67951810 -0.25937605 -0.62096345  0.68446565;;;;]
tmp_β = β .- μ ./ sqrt.(σ² .+ ϵ) .* γ
tmp_γ = γ ./ sqrt.(σ² .+ ϵ)
println(typeof(tmp_γ))
tmp_γ = reshape(tmp_γ, (channels, 1))
for i in 1:(ndims(last_A)-2)
    tmp_γ = reshape(tmp_γ, (1, size(tmp_γ)...))
end
next_A = last_A .* tmp_γ
channel_dim = ndims(last_A) - 1
if (ndims(last_A) > 2)
    tmp_last_A = dropdims(sum(last_A, dims = Tuple(1:channel_dim-1)), dims = Tuple(1:channel_dim-1))
    sum_bias = dropdims(sum((tmp_last_A .* tmp_β), dims = 1), dims = 1)
else
    sum_bias = dropdims(sum((last_A .* tmp_β), dims = channel_dim), dims = channel_dim)
end



                    #batch_reach, batch_reach_β, info = backward_linear(bn_layer, input_size, batch_output, nothing)
#print(batch_reach_β)

Vector{Float32}


1-element Vector{Float64}:
 0.0